# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import roc_auc_score

df = pd.read_csv("creditcard.csv")

print("The total length of the dataframe is " + str(len(df.columns)))

n_replicas = 10

big_raw_data = pd.DataFrame(np.repeat(df.values, n_replicas, axis = 0), columns = df.columns)

unique = big_raw_data["Class"].unique()

sizes = big_raw_data["Class"].value_counts().values

fig, ax = plt.subplots()
ax.pie(sizes, labels=unique, autopct='%1.1f%%')
ax.set_title('Piece Counts')
plt.show()

fig2, wx = plt.subplots()
wx.hist(big_raw_data.Amount.values,6, histtype="bar", color = 'blue')
wx.set_title('Piece Counts 2')
wx.set_xlabel('Amount')
wx.set_ylabel('Frequency')
plt.show()  

print("the maximum value you can find here is " + str(big_raw_data.Amount.max()))
print("the 90th percentile is " + str(np.percentile(big_raw_data.Amount.values, 90)))

big_raw_data.iloc[:, 1:30] = StandardScaler().fit_transform(big_raw_data.iloc[:, 1:30])

data_matrix = big_raw_data.values

X = data_matrix[:, 1:30]
y = data_matrix[:, 30]

X = normalize(X, norm='l1')

print('X.shape=', X.shape, 'y.shape=', y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

w_train = compute_sample_weight("balanced", y_train)

from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=4, random_state=35)

print("Original dataset size:", len(big_raw_data))
print("Training set size:", len(x_train))
print("Testing set size:", len(x_test))

t0 = time.time()
model.fit(x_train, y_train, sample_weight=w_train)
sklearn_model_time = time.time() - t0
print("time taken by the model to be trained is ", sklearn_model_time)

sklearn_predict = model.predict_proba(x_test)[:, 1]

sklearn_roc_auc = roc_auc_score(y_test, sklearn_predict)

print("sklearn roc auc score is ", sklearn_roc_auc)

from sklearn.svm import LinearSVC

Linearmodel = LinearSVC(class_weight='balanced', random_state=35, loss='hinge', fit_intercept=False, max_iter=500)

t2 = time.time()
Linearmodel.fit(x_train, y_train)
sklinear_model_time = time.time() - t2
print("time taken by the model to be trained is ", sklinear_model_time)

def tocsv(model, validation_data):
    predictions = model.predict_proba(validation_data)[:, 1]
    return pd.DataFrame({
        "Index" : np.arange(len(predictions)),
        "Prediction" : predictions

    })

validation_predictions = tocsv(model, x_test)
validation_predictions.to_csv("predicted_credit_defaulters_DecisionTreeClassifier.csv", index=False)
print("Decision Tree Classifier predictions saved in csv file.")

sklinear_predict = Linearmodel.decision_function(x_test)
skLinear_roc_auc = roc_auc_score(y_test, sklinear_predict)
print("sklearn roc auc score is ", skLinear_roc_auc)

def tocsvforsvc(Linearmodel, validation_data):
    predictions = Linearmodel.decision_function(validation_data)
    return pd.DataFrame({
        "index" : np.arange(len(predictions)),
        "predictions" : predictions
})

LinearSVCfile = tocsvforsvc(Linearmodel, x_test)
LinearSVCfile.to_csv("predicted_credit_defaulters_LinearSVC.csv", index=False)


print("Linear SVC predictions saved in csv file.")